In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import env

In [2]:
url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/logs'
df = pd.read_sql('SELECT * FROM api_access', url)

In [3]:
df = df.entry.str.split(expand=True)

In [4]:
df[11] = df[11].str.strip('"')

In [5]:
df[11] = df.loc[:,11:].apply(lambda rows: ' '.join(rows.values.astype(str)), axis=1).str.replace('None', '').str.strip()

In [6]:
df = df.loc[:, :11]
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,97.105.19.58,-,-,[16/Apr/2019:19:34:42,+0000],"""GET",/api/v1/sales?page=81,"HTTP/1.1""",200,512495,"""-""",python-requests/2.21.0
1,97.105.19.58,-,-,[16/Apr/2019:19:34:42,+0000],"""GET",/api/v1/items,"HTTP/1.1""",200,3561,"""-""",python-requests/2.21.0
2,97.105.19.58,-,-,[16/Apr/2019:19:34:44,+0000],"""GET",/api/v1/sales?page=82,"HTTP/1.1""",200,510103,"""-""",python-requests/2.21.0
3,97.105.19.58,-,-,[16/Apr/2019:19:34:46,+0000],"""GET",/api/v1/sales?page=83,"HTTP/1.1""",200,510003,"""-""",python-requests/2.21.0
4,97.105.19.58,-,-,[16/Apr/2019:19:34:48,+0000],"""GET",/api/v1/sales?page=84,"HTTP/1.1""",200,511963,"""-""",python-requests/2.21.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13969,97.105.19.58,-,-,[17/Apr/2019:12:55:12,+0000],"""GET",/api/v1/sales?page=62,"HTTP/1.1""",200,511544,"""-""",python-requests/2.21.0
13970,97.105.19.58,-,-,[17/Apr/2019:12:55:12,+0000],"""GET",/api/v1/sales?page=65,"HTTP/1.1""",200,511201,"""-""",python-requests/2.21.0
13971,97.105.19.58,-,-,[17/Apr/2019:12:55:13,+0000],"""GET",/api/v1/sales?page=73,"HTTP/1.1""",200,511781,"""-""",python-requests/2.21.0
13972,97.105.19.58,-,-,[17/Apr/2019:12:55:13,+0000],"""GET",/api/v1/sales?page=69,"HTTP/1.1""",200,510136,"""-""",python-requests/2.21.0


In [7]:
df = df.drop(columns=[1,2,4,10])

In [8]:
df = df.rename(columns={0:'ip',
                  3:'timestamp',
                  5:'request_method',
                  6:'request_path',
                  7:'http_version',
                  8:'status_code',
                  9:'size',
                  11:'user_agent'})

In [9]:
df.request_method = df.request_method.str.strip('"')
df.http_version = df.http_version.str.strip('"')

In [10]:
df.timestamp = df.timestamp.str.strip('[').str.replace(':', ' ', 1)

In [11]:
new = pd.DataFrame([
    ["95.31.18.119", "21/Apr/2019 10:02:41", "GET", "/api/v1/items/", "HTTP/1.1", '200', 1153005, "python-requests/2.21.0"],
    ["95.31.16.121", "17/Apr/2019 19:36:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '301', 1005, "python-requests/2.21.0"],
    ["97.105.15.120", "18/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '301', 2560, "python-requests/2.21.0"],
    ["97.105.19.58", "19/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '200', 2056327, "python-requests/2.21.0"],
], columns=df.columns)

new

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,95.31.18.119,21/Apr/2019 10:02:41,GET,/api/v1/items/,HTTP/1.1,200,1153005,python-requests/2.21.0
1,95.31.16.121,17/Apr/2019 19:36:41,GET,/api/v1/sales?page=79/,HTTP/1.1,301,1005,python-requests/2.21.0
2,97.105.15.120,18/Apr/2019 19:42:41,GET,/api/v1/sales?page=79/,HTTP/1.1,301,2560,python-requests/2.21.0
3,97.105.19.58,19/Apr/2019 19:42:41,GET,/api/v1/sales?page=79/,HTTP/1.1,200,2056327,python-requests/2.21.0


In [13]:
df = pd.concat([df, new]).reset_index(drop=True)

In [14]:
df.timestamp = pd.to_datetime(df.timestamp)
df

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,2019-04-16 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,2019-04-16 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,2019-04-16 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,2019-04-16 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,2019-04-16 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0
...,...,...,...,...,...,...,...,...
13973,97.105.19.58,2019-04-17 12:55:14,GET,/api/v1/sales?page=71,HTTP/1.1,200,510166,python-requests/2.21.0
13974,95.31.18.119,2019-04-21 10:02:41,GET,/api/v1/items/,HTTP/1.1,200,1153005,python-requests/2.21.0
13975,95.31.16.121,2019-04-17 19:36:41,GET,/api/v1/sales?page=79/,HTTP/1.1,301,1005,python-requests/2.21.0
13976,97.105.15.120,2019-04-18 19:42:41,GET,/api/v1/sales?page=79/,HTTP/1.1,301,2560,python-requests/2.21.0


In [18]:
df['request_path'].value_counts()

/api/v1/sales?page=2               709
/api/v1/items                      464
/api/v1/items?page=2               291
/api/v1/items?page=3               219
/api/v1/stores                     162
                                  ... 
/api/v1                              1
/api/v1//api/v1/items/next_page      1
/api/v1//api/v1/items                1
/api/v1/items?page=0                 1
/api/V1/HiZach!                      1
Name: request_path, Length: 220, dtype: int64

In [19]:
df['user_agent'].value_counts()

python-requests/2.21.0                                                                                                        12005
python-requests/2.20.1                                                                                                         1911
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36"       34
Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"                                               8
Slackbot-LinkExpanding 1.0 (+https://api.slack.com/robots)"                                                                       7
Slackbot 1.0 (+https://api.slack.com/robots)"                                                                                     6
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36"        4
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, l